In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from backtesting import Backtest, Strategy 
from backtesting.lib import crossover
from backtesting.test import SMA

/opt/conda/lib/python3.9/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
import talib

In [2]:
hist = yf.download(
    tickers="AAPL MSFT GOOG",
    period="2y",
    group_by="ticker"
)

[*********************100%***********************]  3 of 3 completed


In [3]:
hist

GOOG                                                      \
                   Open         High          Low        Close    Adj Close   
Date                                                                          
2019-06-17  1086.280029  1099.180054  1086.280029  1092.500000  1092.500000   
2019-06-18  1109.689941  1116.390015  1098.989990  1103.599976  1103.599976   
2019-06-19  1105.599976  1107.000000  1093.479980  1102.329956  1102.329956   
2019-06-20  1119.989990  1120.119995  1104.739990  1111.420044  1111.420044   
2019-06-21  1109.239990  1124.109985  1108.079956  1121.880005  1121.880005   
...                 ...          ...          ...          ...          ...   
2021-06-11  2524.919922  2526.989990  2498.290039  2513.929932  2513.929932   
2021-06-14  2513.389893  2528.229980  2500.939941  2527.040039  2527.040039   
2021-06-15  2530.439941  2537.239990  2512.969971  2520.659912  2520.659912   
2021-06-16  2524.949951  2530.469971  2482.999023  2513.929932  2513.929932   
2021-06-17  2510.459961  2521.294922  2510.300049  2520.429932  2520.429932   

                           MSFT                                      \
             Volume        Open        High         Low       Close   
Date                                                                  
2019-06-17   941600  132.630005  133.729996  132.529999  132.850006   
2019-06-18  1386700  134.190002  135.240005  133.570007  135.160004   
2019-06-19  1338800  135.000000  135.929993  133.809998  135.690002   
2019-06-20  1262000  137.449997  137.660004  135.720001  136.949997   
2019-06-21  1947600  136.580002  137.729996  136.460007  136.970001   
...             ...         ...         ...         ...         ...   
2021-06-11  1262000  257.989990  258.489990  256.609985  257.890015   
2021-06-14  1127500  257.899994  259.950012  256.799988  259.890015   
2021-06-15  1109100  259.769989  259.989990  257.679993  258.359985   
2021-06-16  1311100  259.399994  260.579987  254.419998  257.380005   
2021-06-17    95946  256.065002  257.848999  256.010010  257.838501   

                                        AAPL                          \
             Adj Close    Volume        Open        High         Low   
Date                                                                   
2019-06-17  129.976608  14517800   48.224998   48.740002   48.042500   
2019-06-18  132.236633  25934500   49.012501   50.072498   48.802502   
2019-06-19  132.755188  23744400   49.919998   49.970001   49.327499   
2019-06-20  133.987885  33042600   50.092499   50.152500   49.507500   
2019-06-21  134.007477  36727900   49.700001   50.212502   49.537498   
...                ...       ...         ...         ...         ...   
2021-06-11  257.890015  18995100  126.529999  127.440002  126.099998   
2021-06-14  259.890015  19150500  127.820000  130.539993  127.070000   
2021-06-15  258.359985  18038900  129.940002  130.600006  129.389999   
2021-06-16  257.380005  27178100  130.369995  130.889999  128.460007   
2021-06-17  257.838501   2256119  129.800003  130.809998  129.649994   

                                               
                 Close   Adj Close     Volume  
Date                                           
2019-06-17   48.472500   47.580223   58676400  
2019-06-18   49.612499   48.699249  106204000  
2019-06-19   49.467499   48.556911   84496800  
2019-06-20   49.865002   48.947098   86056000  
2019-06-21   49.695000   48.780224  191202400  
...                ...         ...        ...  
2021-06-11  127.349998  127.349998   53415300  
2021-06-14  130.479996  130.479996   96906500  
2021-06-15  129.639999  129.639999   62746300  
2021-06-16  130.149994  130.149994   91657900  
2021-06-17  130.690002  130.690002    9801091  

[506 rows x 18 columns]

In [12]:
hist["AAPL"]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-06-17,48.224998,48.740002,48.042500,48.472500,47.580223,58676400
2019-06-18,49.012501,50.072498,48.802502,49.612499,48.699249,106204000
2019-06-19,49.919998,49.970001,49.327499,49.467499,48.556911,84496800
2019-06-20,50.092499,50.152500,49.507500,49.865002,48.947098,86056000
2019-06-21,49.700001,50.212502,49.537498,49.695000,48.780224,191202400
...,...,...,...,...,...,...
2021-06-10,127.019997,128.190002,125.940002,126.110001,126.110001,71186400
2021-06-11,126.529999,127.440002,126.099998,127.349998,127.349998,53415300
2021-06-14,127.820000,130.539993,127.070000,130.479996,130.479996,96906500
